# Generation Module Verification

This notebook verifies the functionality of **ALL** components in `src.generation`:
1.  **Generator** (Llama 3.1 8B or 3.2 3B 4-bit)
2.  **Query Rewriter**
3.  **Retrieval Grader** (CRAG)
4.  **Hallucination Grader** (Self-RAG)

## Setup and Imports
First, we set up the environment and import necessary libraries. 
We append the project root to `sys.path` to allow importing modules from `src/`.

In [5]:
import sys
import os
# from tqdm.notebook import tqdm # Use this if strict notebook, but standard print is safer for mixed envs

# Ensure we can import from src
# Assuming this notebook is running from 'tests/'
project_root = os.path.abspath(os.path.join(os.getcwd(), '..'))
if project_root not in sys.path:
    sys.path.append(project_root)

print(f"Project Root added: {project_root}")

from src.generation import create_generation_components

Project Root added: /home/marcantoniolopez/Documenti/github/projects/llm-semeval-task8


## Configuration
Select the model ID to use. 
- Use `meta-llama/Llama-3.2-3B-Instruct` for faster local testing.
- Use `meta-llama/Meta-Llama-3.1-8B-Instruct` for production-grade testing.

In [6]:
# CONFIGURATION
# MODEL_ID = "meta-llama/Meta-Llama-3.1-8B-Instruct" # Standard
MODEL_ID = "meta-llama/Llama-3.2-3B-Instruct" # Light for local testing

## Helper Classes
Define helper classes for colored terminal output to make the verification results easier to read.

In [7]:
# --- HELPER CLASS FOR STYLED OUTPUT ---

class Colors:
    HEADER = '\033[95m'
    BLUE = '\033[94m'
    CYAN = '\033[96m'
    GREEN = '\033[92m'
    YELLOW = '\033[93m'
    RED = '\033[91m'
    BOLD = '\033[1m'
    UNDERLINE = '\033[4m'
    RESET = '\033[0m'

def print_header(title):
    print(f"\n{Colors.HEADER}{Colors.BOLD}{'='*60}{Colors.RESET}")
    print(f"{Colors.HEADER}{Colors.BOLD} {title} {Colors.RESET}")
    print(f"{Colors.HEADER}{Colors.BOLD}{'='*60}{Colors.RESET}\n")

def print_section(title):
    print(f"\n{Colors.CYAN}{Colors.BOLD}>>> {title} {Colors.RESET}")

def print_input(label, content):
    print(f"{Colors.YELLOW}{Colors.BOLD}{label}:{Colors.RESET} {content}")

def print_output(output, expected):
    print(f"{Colors.GREEN}{Colors.BOLD}Output:{Colors.RESET} {output}")
    print(f"{Colors.BLUE}{Colors.BOLD}Expected:{Colors.RESET} {expected}")

## Component Initialization
Initialize the generation components using the factory function. This loads the model (quantized) and sets up the LangChain pipelines.

In [8]:
print_header("INITIALIZATION")
components = create_generation_components(model_id=MODEL_ID)


 INITIALIZATION 

Creating Generation Components with model: meta-llama/Llama-3.2-3B-Instruct...


Loading checkpoint shards: 100%|██████████| 2/2 [00:04<00:00,  2.14s/it]
Device set to use cuda:0


Generation Components Ready.


## 1. Test Generator
The Generator is responsible for answering questions based on the provided context.

### 1.1 Positive Case
We provide a context that contains the answer. The model should correctly extract the information.

In [9]:
print_section("TEST: Generator (Positive)")
context = "Tim Cook is the CEO of Apple."
question = "Who is the CEO of Apple?"

print_input("Context", context)
print_input("Question", question)

res = components.generator.invoke({"context": context, "question": question})
print_output(res.strip(), "Something like 'Tim Cook'")


>>> TEST: Generator (Positive) 
Context: Tim Cook is the CEO of Apple.
Question: Who is the CEO of Apple?
Output: Tim Cook
Expected: Something like 'Tim Cook'


### 1.2 Negative Case (Fallback)
We provide a context that is irrelevant to the question. The model **must** strictly reply with `I_DONT_KNOW`.

In [10]:
print_section("TEST: Generator (Negative)")
context = "The sky is blue."
question = "Who is the CEO of Apple?"

print_input("Context", context)
print_input("Question", question)

res = components.generator.invoke({"context": context, "question": question})
print_output(res.strip(), "'I_DONT_KNOW'")


>>> TEST: Generator (Negative) 
Context: The sky is blue.
Question: Who is the CEO of Apple?
Output: I_DONT_KNOW
Expected: 'I_DONT_KNOW'


## 2. Test Query Rewriter
The Query Rewriter transforms a user question that might depend on previous chat history (e.g., "When was *he* born?") into a standalone question.

In [11]:
print_section("TEST: Query Rewriter")
history = [
    ("human", "Who is the creator of Python?"),
    ('assistant', 'Guido van Rossum')
]
question = "When was he born?"

print_input("History", str(history))
print_input("Question", question)

res = components.query_rewriter.invoke({"messages": history, "question": question})
print_output(res.strip(), "'When was Guido van Rossum born?'")


>>> TEST: Query Rewriter 
History: [('human', 'Who is the creator of Python?'), ('assistant', 'Guido van Rossum')]
Question: When was he born?
Output: When was Guido van Rossum born?
Expected: 'When was Guido van Rossum born?'


## 3. Test Retrieval Grader (CRAG)
This component evaluates whether a retrieved document is relevant to the question. It returns a JSON with a binary score.

### 3.1 Relevant Document

In [12]:
print_section("TEST: Retrieval Grader (Relevant)")
question = "What is the capital of France?"
document = "Paris is the capital of France."

print_input("Question", question)
print_input("Document", document)

res = components.retrieval_grader.invoke({"question": question, "document": document})
print_output(res, "{'binary_score': 'yes'}")


>>> TEST: Retrieval Grader (Relevant) 
Question: What is the capital of France?
Document: Paris is the capital of France.
Output: {'binary_score': 'yes'}
Expected: {'binary_score': 'yes'}


### 3.2 Irrelevant Document

In [13]:
print_section("TEST: Retrieval Grader (Irrelevant)")
question = "What is the capital of France?"
document = "To make pizza you need flour."

print_input("Question", question)
print_input("Document", document)

res = components.retrieval_grader.invoke({"question": question, "document": document})
print_output(res, "{'binary_score': 'no'}")


>>> TEST: Retrieval Grader (Irrelevant) 
Question: What is the capital of France?
Document: To make pizza you need flour.
Output: {'binary_score': 'no'}
Expected: {'binary_score': 'no'}


## 4. Test Hallucination Grader (Self-RAG)
This component verifies if the generated answer is supported by the retrieved facts (groundedness).

### 4.1 Grounded Answer

In [14]:
print_section("TEST: Hallucination Grader (Grounded)")
documents = "The Python programming language was created by Guido van Rossum."
generation = "Guido van Rossum created Python."

print_input("Documents", documents)
print_input("Generation", generation)

res = components.hallucination_grader.invoke({"documents": documents, "generation": generation})
print_output(res, "{'binary_score': 'yes'}")


>>> TEST: Hallucination Grader (Grounded) 
Documents: The Python programming language was created by Guido van Rossum.
Generation: Guido van Rossum created Python.
Output: {'binary_score': 'yes'}
Expected: {'binary_score': 'yes'}


### 4.2 Hallucinated Answer
The answer is not supported by the document provided.

In [15]:
print_section("TEST: Hallucination Grader (Hallucination)")
documents = "The Python programming language was created by Guido van Rossum."
generation = "Elon Musk created Python."

print_input("Documents", documents)
print_input("Generation", generation)

res = components.hallucination_grader.invoke({"documents": documents, "generation": generation})
print_output(res, "{'binary_score': 'no'}")


>>> TEST: Hallucination Grader (Hallucination) 
Documents: The Python programming language was created by Guido van Rossum.
Generation: Elon Musk created Python.
Output: {'binary_score': 'no'}
Expected: {'binary_score': 'no'}
